In [5]:
import h5py

def print_shapes(h5obj, prefix=""):
    for key in h5obj:
        item = h5obj[key]
        if isinstance(item, h5py.Dataset):
            print(f"{prefix}{key}: shape {item.shape}, dtype {item.dtype}")
        elif isinstance(item, h5py.Group):
            print(f"{prefix}{key}/ (Group)")
            print_shapes(item, prefix=prefix + "  ")

mat_path = r"/Users/avikpamarthi/Documents/VS Code/FUSI/fusiData/doppler/doppler_S27_R99+normcorre.mat"
with h5py.File(mat_path, 'r') as f:
    print_shapes(f)

#refs#/ (Group)
  0: shape (1, 1), dtype float64
  01: shape (1, 1), dtype uint8
  02: shape (1, 1), dtype uint8
  03: shape (1, 1), dtype uint8
  04: shape (6, 1), dtype uint16
  05: shape (6, 1), dtype uint16
  06: shape (6, 1), dtype uint16
  07: shape (6, 1), dtype uint16
  0A: shape (12, 1), dtype uint16
  0B: shape (12, 1), dtype uint16
  0C: shape (12, 1), dtype uint16
  0D: shape (12, 1), dtype uint16
  0E: shape (12, 1), dtype uint16
  0F: shape (12, 1), dtype uint16
  0G: shape (12, 1), dtype uint16
  0H: shape (12, 1), dtype uint16
  0I: shape (12, 1), dtype uint16
  0J: shape (12, 1), dtype uint16
  0K: shape (12, 1), dtype uint16
  0L: shape (12, 1), dtype uint16
  0M: shape (12, 1), dtype uint16
  0N: shape (1, 1), dtype float64
  0O: shape (1, 1), dtype float64
  0P: shape (1, 1), dtype float64
  0Q: shape (1, 1), dtype float64
  0R: shape (1, 1), dtype object
  0S: shape (1, 1), dtype object
  0T: shape (1, 1), dtype object
  0U: shape (1, 1), dtype object
  0V: shape (

In [7]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import h5py
from scipy.io import loadmat

def load_iDop(path):
    """Load iDop from a .mat (v7.3 or older) as a 4D float32 NumPy array."""
    try:
        mat = loadmat(path)
        arr = mat['iDop']
    except NotImplementedError:
        with h5py.File(path, 'r') as f:
            arr = f['iDop'][()]   # shape: (trials, slow_time, H, W)
    return np.ascontiguousarray(arr, dtype=np.float32)

# 1) point to your folder and grab two files
DATA_DIR = "/Users/avikpamarthi/Documents/VS Code/FUSI/fusiData/doppler"
all_mat = sorted(f for f in os.listdir(DATA_DIR) if f.lower().endswith('.mat'))
paths = [os.path.join(DATA_DIR, fn) for fn in all_mat[:2]]

# 2) load into a list of 4D torch tensors
tensors = []
for p in paths:
    arr4 = load_iDop(p)                   # e.g. (242, 15, 128, 103)
    tensors.append(torch.from_numpy(arr4))

# 3) find the max size along each of the 4 dims
ndim = tensors[0].ndim  # should be 4
max_sizes = [ max(t.shape[d] for t in tensors) for d in range(ndim) ]
# e.g. max_sizes == [max_trials, max_slow, max_H, max_W]

# 4) pad each tensor up to (max_trials, max_slow, H, W)
padded = []
for t in tensors:
    # A) pad axis=0 (trials) by concat, if needed
    if t.shape[0] < max_sizes[0]:
        pad0 = torch.zeros((max_sizes[0] - t.shape[0], *t.shape[1:]),
                           dtype=t.dtype, device=t.device)
        t = torch.cat([t, pad0], dim=0)

    # B) pad the remaining axes (1,2,3) via F.pad
    #    F.pad wants pads for the *last* dims first: (W_left, W_right, H_left, H_right, slow_left, slow_right)
    pads = []
    for d in reversed(range(1, ndim)):
        diff = max_sizes[d] - t.shape[d]
        pads.extend([0, diff])   # (pad_left=0, pad_right=diff)
    t = F.pad(t, pads, mode='constant', value=0)

    padded.append(t)

# 5) stack into (n_files, max_trials, max_slow, max_H, max_W)
X = torch.stack(padded, dim=0)
print("Final shape:", X.shape)
# -> e.g. torch.Size([2, 242, 15, 128, 103])
print(X)

Final shape: torch.Size([2, 12135, 128, 143])
tensor([[[[4.9815e+10, 4.9815e+10, 4.9815e+10,  ..., 4.9815e+10,
           4.9815e+10, 4.9815e+10],
          [4.9815e+10, 2.9608e+13, 3.9244e+13,  ..., 1.7110e+11,
           1.9923e+11, 4.9815e+10],
          [4.9815e+10, 5.3796e+13, 7.1038e+13,  ..., 1.8070e+11,
           2.1446e+11, 4.9815e+10],
          ...,
          [4.9815e+10, 1.9450e+13, 1.9767e+13,  ..., 1.7176e+11,
           1.7645e+11, 4.9815e+10],
          [4.9815e+10, 1.5773e+13, 1.6514e+13,  ..., 1.2756e+11,
           1.5708e+11, 4.9815e+10],
          [4.9815e+10, 7.7306e+12, 8.9338e+12,  ..., 1.0273e+11,
           1.3055e+11, 4.9815e+10]],

         [[4.7716e+10, 4.7716e+10, 4.7716e+10,  ..., 4.7716e+10,
           4.7716e+10, 4.7716e+10],
          [4.7716e+10, 3.6023e+13, 3.9102e+13,  ..., 1.5826e+11,
           1.3638e+11, 4.7716e+10],
          [4.7716e+10, 6.1380e+13, 7.0832e+13,  ..., 1.6098e+11,
           1.3701e+11, 4.7716e+10],
          ...,
          [4.